# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import norm
from statsmodels.stats.proportion import proportions_ztest

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
print('# of callbacks for black-sounding names: ', sum(data[data.race=='b'].call))
#number of callbacks for white-sounding names
print('# of callbacks for white-sounding names: ', sum(data[data.race=='w'].call))

# of callbacks for black-sounding names:  157.0
# of callbacks for white-sounding names:  235.0


In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

In [6]:
data.shape

(4870, 65)

In [7]:
#Calculate sample proportions
w = data[data.race=='w']
b = data[data.race=='b']

#sample proportion of resumes with white-sounding names that got callbacks
p_hat_w = sum(w.call)/len(w.call)

#sample proportion of resumes with black-sounding names that got callbacks
p_hat_b = sum(b.call)/len(w.call)

#sample proportion of resumes that got callbacks - pooled 
pooled_p = sum(data.call)/len(data.call)

print('Sample Size: ', len(data.call))
print('Sample size of white-sounding names: ', len(w.call))
print('Sample size of black-sounding names: ', len(b.call))
print('Sample callback rate for white-sounding names: ', p_hat_w)
print('Sample callback rate black-sounding names: ', p_hat_b)
print('Pooled sample callback rate: ', pooled_p)

Sample Size:  4870
Sample size of white-sounding names:  2435
Sample size of black-sounding names:  2435
Sample callback rate for white-sounding names:  0.09650924024640657
Sample callback rate black-sounding names:  0.06447638603696099
Pooled sample callback rate:  0.08049281314168377


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>   
</div>

## 1. What test is appropriate for this problem? Does CLT apply?

In this problem, a two sample z-test is apporopriate.  I will use a hypothesis test for proportions because callback rates are proportions. <br>

The Central Limit Theorem does apply. <br>
The trials are independent.  They were also random since the resumes were randomly assigned either a black-sounding or white-sounding name.  Also the sample size is n = 4870, 2435 black-sounding names and 2435 white-sounding names  and the number of successes (callback, in this case) and the number of failures (no callback) are both epected to be greater than 10 for both white-sounding names and black sounding-names.  

## 2. What are the null and alternate hypotheses?

Null hypothesis: There is no difference in callback rates for job applicants with white-sounding names and job applicants with black-sounding names. <br>
$H_{0}$: $p_{w}$ - $p_{b} $= 0<br>
Alternate hypothesis: There is a difference in callback rates for white-sounding names and black-sounding names. <br>
$H_{A}$: $p_{w}$ - $p_{b}$ $\neq$ 0

$\alpha$ = 0


In [8]:
# Your solution to Q3 here

## 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [9]:
# bootstrapping 

#simulate data assuming the null hypothesis that callback rates are equal for white and black sounding names
bs_reps = np.empty(100000)
for i in range(100000):
    bs_sample_w = np.random.binomial(n=1, p = pooled_p, size = len(w.call))
    bs_sample_b = np.random.binomial(n=1, p = pooled_p, size = len(b.call))
    bs_reps[i] = sum(bs_sample_w)/len(bs_sample_w) - sum(bs_sample_b)/len(bs_sample_b)

diff_obs = p_hat_w - p_hat_b
print('Observed difference: ', diff_obs)


#95% confidence interval    
bs_confint = np.percentile(bs_reps, [2.5, 97.5])
print('Bootstrap 95% confidence interval: ', bs_confint)

#p-value
p_val = np.sum(bs_reps >= diff_obs)/100000
print('The p-value is ', p_val)

Observed difference:  0.032032854209445585
Bootstrap 95% confidence interval:  [-0.01519507  0.01519507]
The p-value is  1e-05


Given that the observed difference between callback rates for white-sounding and black-sounding names (0.032) falls outside of the bootstrap confidence interval and that the p-value is much smaller than 0.05, we are can reject the null hypothesis that the callback rates are the same.

In [10]:
#frequentist approach

z_star = norm.ppf(0.975)

moe = z_star * np.sqrt((pooled_p)*(1-pooled_p)/len(data.race))
print('margin of error: ', moe)

conf_int = (diff_obs - moe,  diff_obs + moe)
print('95% confindence interval: ', conf_int)

count = [sum(w.call), sum(b.call)]
nobs = [len(w.call), len(b.call)]
z, p_z = proportions_ztest(count, nobs)


print('p-value: ', p_z)


margin of error:  0.007640815751084616
95% confindence interval:  (0.02439203845836097, 0.0396736699605302)
p-value:  3.983886837585077e-05


We are 95% confident that the difference in callbackrate for white-sounding and black-sounding names is between 0.0244 and 0.0397 higher.
Based on the small p-value, there is a siginificant difference between the call back rates for white-sounding names and black-sounding names.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

## 4. Write a story describing the statistical significance in the context or the original problem.

Based on the data from the experiment, race is a factor in callback rates for job applicants who send in resumes, and there is still racial discrimination in the job market. People with white-sounding names are more likely to hear back from employers. <br>
The proportion of white callback rates is between 0.0244 and 0.0397 greater than the proportion of black callback rates.  While these may not seem like a big difference, putting it in the context of the problem, we see that it is.
In fact, in the sample, white-sounding names had a 9.65% callback rate and black-sounding names 6.45% callback rate.  This means that people with white-sounding names had 49.6% ((9.65-6.45)/6.45) more callbacks than people with black-sounding names.

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

While the above analysis does suggest that race/name plays a role in callback success, it does not mean that race/name is the most important factor in callback success.  The analysis suggests that if there were two identical resumes, one with a white-sounding name and one with a black-sounding name, the white-sounding name would be more likely to receive a callback.  However, there were many other variables measured in the study, such as years of experience, education, skills, sex, etc. that we did not consider in the analysis. 

I would amend the analysis by varying different factors such as education and sex.  

